# Assignment description

Build synthetic scale-free networks according to the BA model. Consider $m$ new connections for each node inserted. Calculate empiracally properties like CC and APL. Compare the obtained results for networks of same size generated by Poisson model and WS model. What do you observe?

In [2]:
%config Completer.use_jedi = False

In [3]:
import igraph as ig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Graph parameters

We're going to build some Erdos-Renyi (ER) models varying its parameters. When we fix a connection probability we're also fixing its degree. So we'll create some realizations for each combination of n and z.

In [62]:
nodes_array = np.arange(1000, 12000, 2000)#[1000, 2000, 4000, 6000, 8000, 10000]
avg_degree_array = [0, 0.5, 0.9, 1, 2, 3, 4, "2log"] #np.arange(0, 3.2, 0.2)
ensemble_size = 2

In [58]:
len(nodes_array) * len(avg_degree_array ) * ensemble_size

96

In [5]:
def find_percolation_threshold(n = 1000, threshold = 0.5, step = 0.2):
    graphs_dict = dict()
    avg_degree = 0
    while True:
        nodes = n
        graphs_dict[avg_degree] = dict()

        probability = avg_degree/nodes        
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_dict[avg_degree][probability] = graph
        giant = graph.clusters().giant()
        graphs_dict[avg_degree][probability]["giant"] = giant
        giant_fraction = giant.vcount()/n
        avg_degree += step
        if giant_fraction > threshold:
            return graphs_dict

percolations = find_percolation_threshold()

In [6]:
percolations

{0: {0.0: <igraph.Graph at 0x7fea9babf350>},
 0.2: {0.0002: <igraph.Graph at 0x7fea9babf450>},
 0.4: {0.0004: <igraph.Graph at 0x7fea9babf650>},
 0.6000000000000001: {0.0006000000000000001: <igraph.Graph at 0x7fea9babf850>},
 0.8: {0.0008: <igraph.Graph at 0x7fea9babfa50>},
 1.0: {0.001: <igraph.Graph at 0x7fea9babfc50>},
 1.2: {0.0012: <igraph.Graph at 0x7fea9babfe50>},
 1.4: {0.0014: <igraph.Graph at 0x7fea9bb67150>}}

## Graph building

### Poisson networks

In [63]:
# %%time
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of nodes and probabilities
perm = product(*[nodes_array, avg_degree_array])

poisson_graphs = dict()
# Print the obtained permutations
for network_property in list(perm):
    
    graphs_list = list()
    nodes = int(network_property[0])
    
    if network_property[1] in ["log", "2log"]:
        degree = 2*np.round(np.log(nodes), 2)
        key = str(nodes) + '_nodes__' + network_property[1] + "_degree"
    else:
        degree = network_property[1]
        key = str(nodes) + '_nodes__' + str(degree) + "_degree"
    probability = degree/nodes
    
    # Create a few realizations
    for i in range(ensemble_size):
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_list.append(graph)
    
    poisson_graphs[key] = dict()
    poisson_graphs[key]['graphs'] = graphs_list
    poisson_graphs[key]["nodes"] = nodes
    poisson_graphs[key]["degree"] = network_property[1]
    poisson_graphs[key]["probability"] = probability    

In [66]:
%%time
# Calculate some metrics for each graph realization

nodes_number = list()
edges_number = list()
clustering_coefficient = list()
avg_path_length = list()

for params in poisson_graphs.keys():
    for graph in poisson_graphs[params]["graphs"]:
        clustering_coefficient.append(graph.transitivity_undirected())
        avg_path_length.append(graph.average_path_length())
        edges_number.append(len(graph.get_edgelist()))
        nodes_number.append(poisson_graphs[params]["nodes"])        

CPU times: user 1min 32s, sys: 801 ms, total: 1min 33s
Wall time: 1min 42s


In [69]:
poisson_analysis = pd.DataFrame()

poisson_analysis["nodes_number"] = nodes_number
poisson_analysis["edges_number"] = edges_number
poisson_analysis["clustering_coefficient"] = clustering_coefficient
poisson_analysis["avg_path_length"] = avg_path_length

poisson_analysis.head()

,nodes_number,edges_number,clustering_coefficient,avg_path_length
0,1000,0,NaN,NaN
1,1000,0,NaN,NaN
2,1000,226,0.0,1.581395
3,1000,299,0.0,2.037481
4,1000,442,0.0,5.615960


In [74]:
df_plot = poisson_analysis.pivot_table(index = ["nodes_number", "edges_number"],
                                         values = ['avg_path_length', "clustering_coefficient"],
                                         aggfunc = ["mean"])
df_plot

mean                       
                          avg_path_length clustering_coefficient
nodes_number edges_number                                       
1000         226                 1.581395               0.000000
             299                 2.037481               0.000000
             439                 5.445302               0.000000
             442                 5.615960               0.000000
             499                 6.027942               0.000000
...                                   ...                    ...
11000        16640               8.406591               0.000360
             22161               6.793254               0.000234
             22162               6.787575               0.000402
             102111              3.518387               0.001742
             102746              3.511622               0.001701

[84 rows x 2 columns]

### Barabasi-Albert networks

In [40]:
nodes_array = np.arange(1000, 12000, 2000)#[1000, 2000, 4000, 6000, 8000, 10000]
ensemble_size = 2
m_number = np.arange(1, 6, 1)

In [41]:
%%time
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of nodes and probabilities
perm = product(*[nodes_array, m_number])

barabasi_graphs = dict()
# Print the obtained permutations
for network_property in list(perm):
    
    graphs_list = list()
    nodes = int(network_property[0])
    m_value = int(network_property[1])
    
    # Create a few realizations
    for i in range(ensemble_size):
        graph = ig.Graph.Barabasi(n = nodes, m = m_value)

        graphs_list.append(graph)

    key = str(nodes) + "_nodes__" + str(m_value) + "_m"
    barabasi_graphs[key] = dict()
    barabasi_graphs[key]['graphs'] = graphs_list
    barabasi_graphs[key]["nodes"] = nodes
    barabasi_graphs[key]["m"] = m_value

CPU times: user 216 ms, sys: 21.8 ms, total: 238 ms
Wall time: 261 ms


In [75]:
%%time
clustering_coefficient = list()
avg_path_length = list()
nodes_number = list()
edges_number = list()
m_values = list()

for params in barabasi_graphs.keys():
    for graph in barabasi_graphs[params]["graphs"]:
        clustering_coefficient.append(graph.transitivity_undirected())
        avg_path_length.append(graph.average_path_length())
        nodes_number.append(barabasi_graphs[params]["nodes"])
        m_values.append(barabasi_graphs[params]["m"]) 
        edges_number.append(len(graph.get_edgelist()))
        

CPU times: user 1min 24s, sys: 810 ms, total: 1min 25s
Wall time: 1min 31s


In [54]:
barabasi_analysis = pd.DataFrame()
barabasi_analysis["nodes_number"] = nodes_number
barabasi_analysis["m_values"] = m_values
barabasi_analysis["avg_path_length"] = avg_path_length
barabasi_analysis["clustering_coefficient"] = clustering_coefficient

barabasi_analysis.fillna(0, inplace = True)

In [55]:
df_plot = barabasi_analysis.pivot_table(index = ["nodes_number", "m_values"],
                                         values = ['avg_path_length', "clustering_coefficient"],
                                         aggfunc = ["mean"])
df_plot

mean                       
                      avg_path_length clustering_coefficient
nodes_number m_values                                       
0            1               0.000000               0.000000
             2               0.000000               0.000000
             3               0.000000               0.000000
             4               0.000000               0.000000
             5               0.000000               0.000000
2000         1               9.726572               0.000000
             2               4.697403               0.005516
             3               3.825570               0.010915
             4               3.477351               0.014452
             5               3.231817               0.018319
4000         1              10.656093               0.000000
             2               5.021654               0.003133
             3               4.117016               0.006195
             4               3.699782               0.008358
             5               3.456469               0.011002
6000         1              10.705710               0.000000
             2               5.137251               0.002335
             3               4.243168               0.004258
             4               3.846458               0.005886
             5               3.580167               0.007590
8000         1              11.625627               0.000000
             2               5.284695               0.001650
             3               4.401269               0.003280
             4               3.942499               0.004753
             5               3.655206               0.006052
10000        1              11.846846               0.000000
             2               5.360810               0.001822
             3               4.450572               0.002893
             4               4.010929               0.003936
             5               3.731994               0.005136